## Scraping Definitions

In [ ]:
# 有道词典 definitions 
defs = driver.find_element_by_xpath('//*[@id="phrsListTab"]/div[2]/ul/p').text
# LINE Dict. definitions
defs = driver.find_element_by_xpath('//*[@id="content"]/div[1]/div[3]/ol').text

In [1]:
# Import necessary modules
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import re
from io import StringIO
import json
import urllib.request
import pandas as pd
import numpy as np
import playsound
from numpy import random
from datetime import datetime
from langdetect import detect
import winsound
from playsound import playsound
from gtts import gTTS
import os
from datetime import timedelta

## How Do I Study Today?

In [2]:
file = open('what_do.txt', mode = 'r')
what_do = file.read()
file.close()
if what_do == 'listening':
    print("Listening Today")
    f = open('what_do.txt', 'w+')
    f.write('reading')
    f.close()
elif what_do == 'reading':
    print("Reading Today")
    f = open('what_do.txt', 'w+')
    f.write('listening')
    f.close()
else:
    pass

Reading Today


## What Words Do I Use to Write Sentences?

In [3]:
languages = ['中文', '한국어', '日本語']
sentenceDate = datetime.now().date()
for i in languages:
    # Read the Excel files into DataFrames
    wordsDF = pd.read_excel(r'Databases\{}_words.xlsx'.format(i), sheet_name = 'words', header = 0)
    sentencesDF =  pd.read_excel(r'Databases\{}_sentences.xlsx'.format(i), sheet_name = 'sentences', header = 0)
    # Read the Chinese Databases into DataFrames
    words = pd.read_excel(r'Databases\{}_words.xlsx'.format(i), sheet_name = 'words', header = 0)
    # Define the date that we want to retreive
    desired_date = (sentenceDate - timedelta(days = 14)).strftime("%Y-%m-%d")
    # Retrieve the words from the specified date
    write = wordsDF['FOREIGN'].where(words['DATE'] == desired_date)
    # Drop NAN rows
    write = write.dropna().reset_index(drop = True)
    # Display the words with which to write sentences
    print("--------{}--------\n{}\n".format(i, write))

--------中文--------
Series([], Name: FOREIGN, dtype: object)

--------한국어--------
Series([], Name: FOREIGN, dtype: object)

--------日本語--------
Series([], Name: FOREIGN, dtype: object)



## The Importer + Scraper

In [49]:
# Define the current date and time
now = datetime.now().strftime("%Y-%m-%d_%H.%M.%S")
startTime1 = time.time()
# Define the three functions we'll be using
# Define Vocab Notes function
def addVocab(language):
    n = 0
    # Handle the unique 日本語
    if language == '日本語':
    # Loop through all the rows and create a note for each one
        for index, vocab in enumerate(vocabForeign):
            invoke('addNote', note = {
                        "deckName": '{}'.format(language),
                        "modelName": '(NEW) {}'.format(language),
                        "fields": {
                            "(Word) Foreign": vocab, 
                            "(Word) English": vocabEnglish[index],
                            "(Word) Foreign [漢字]": kanji[index]
                            }  
                })
            n += 1
            print("{} has elapsed for {} Vocab words of {} ({} remaining).".format(convert(time.time() - startTime1), n, words.shape[0], words.shape[0] - n))
    # For 한국어 and 中文
    else:
        # Loop through all the rows and create a note for each one
        for index, vocab in enumerate(vocabForeign):
            invoke('addNote', note = {
                        "deckName": '{}'.format(language),
                        "modelName": '(NEW) {}'.format(language),
                        "fields": {
                            "(Word) Foreign": vocab, 
                            "(Word) English": vocabEnglish[index]
                            }  
                })
            n += 1
            print("{} has elapsed for {} Vocab words of {} ({} remaining).".format(convert(time.time() - startTime1), n, words.shape[0], words.shape[0] - n))
# Define Cloze Notes function
def addCloze(language):
    n = 0
    # Loop through all the rows and create a note for each one
    for index, sentence in enumerate(cloze):
        invoke('addNote', note = {
                    "deckName": '{}'.format(language),
                    "modelName": '(NEW) {} Cloze'.format(language),
                    "fields": {
                        "Text": sentence, 
                        "Tip": tip[index],
                        "English": english[index],
                        "full setence": fullSentence[index]
                        }  
            })
        n += 1
        print("{} has elapsed for {} Cloze notes of {} ({} remaining).".format(convert(time.time() - startTime1), n, dfm.shape[0], dfm.shape[0] - n))

# Define the Sentence Scraping function

def scrapeSentences(language):
    words = vocab['Foreign']
    n = 0
    o = words.shape[0]
    # Create master DataFrame
    w = {}
    concatList = []
    # Run the scraper based on the language
    if language == '日本語':
        # Not done
        # See CSV_Gen [ALL].ipynb
        pass
    elif language == '中文':
        # Define URL for Chinese words
        url_zh = 'https://dict.naver.com/linedict/zhendict/dict.html#/cnen/home'
        # Define the driver for Selenium to use
        driver = webdriver.Chrome('chromedriver.exe')
        # Open the specfied URL
        driver.get(url_zh)
        # Find the radio group and select "Examples"
        examples_radio = driver.find_element_by_xpath('//*[@id="ac_form"]/fieldset/ul/li[2]/span').click()
        # Find the Search bar
        search = driver.find_element_by_xpath('//*[@id="ac_input"]')
        # Scrape sentences for each word present in the vocabulary list
        for word in words:
            # Type the word into the Search Box
            search.send_keys(word)
            search.send_keys(Keys.RETURN)
            # Sleep to avoid errors
            time.sleep(3)
            # Find the sentences examples
            examples = driver.find_element_by_xpath('//*[@id="content"]/div[1]/div[2]')
            if examples is not None:
                # Sleep to avoid errors
                time.sleep(1)
                # Define an object with all the example sentences
                unfiltered_sentences = examples.text
                # Clean the string of text that was just scraped
                pass1 = unfiltered_sentences.replace("more", "")
                # Finalize the DataFrame
                myList = pass1.split("\n")
                myList = myList[::2]
                # Drop the 拼音 be creating a dictionary
                myDict = dict(zip(myList[::2], myList[1::2]))
                foreign = myDict.keys()
                english = myDict.values()
                df = pd.DataFrame(columns = ['English', 'Foreign', 'Word', 'Hint', 'Cloze'])
                df['English'] = english
                df['Foreign'] = foreign
                df['Word'] = word
                wordsClean = words.tolist()
                wordsClean = str(wordsClean).strip("[]")
                wordsClean = wordsClean.replace("'", "")
                df['Hint'] = wordsClean
                df['Hint'] = df['Hint'].str.strip('[]')
                df['Hint'] = df['Hint'].str.replace("'", "")
                # Create the Cloze sentences
                df['Cloze'] = df['Foreign'].str.replace(word, '{{c1::' + word + '}}')
                # Remove examples that don't contain a period
                df['English'] = df['English'].map(str)
                df1 = df[df['English'].str.contains('\.', na = False)]
                # Reduce the DataFrame to only 3 sentences
                df2 = df1.iloc[:3]
                # Create the final DataFrame for the word
                w[word] = df2
                # Add the DataFrame to the list of all word DataFrames
                concatList.append(w[word])
                # Sleep to avoid errors
                time.sleep(0.5)
            else:
                pass
            n += 1
            o -= 1
            print("Elapsed time: ({}) {} for {} words ({} remaining)".format(word, convert(time.time() - startTime1), n, o))
            # Exit the FOR loop
        # Quit the browser
        driver.quit()
        # Concatenate each word's DataFrame to the final one with all the sentences
        dfm = pd.concat(concatList, ignore_index = True)
        # Fix formating issues
        dfm['Cloze'] = dfm['Cloze'].str.replace('Name:.*', '')
        dfm['Cloze'] = dfm['Cloze'].str.replace('\n', ',')
        #dfm['Cloze'] = dfm['Cloze'].str.replace('\d', '')
        # return the created DataFrame
        return dfm
    elif language == '한국어':
        # Define URL for Korean words
        url_ko = 'https://en.dict.naver.com/#/search?range=example&shouldSearchVlive=false&query=%ED%9B%84%ED%9A%8C&exampleLevel=exist:1&haveTrans=exist:1'
        # Define the driver for Selenium to use
        driver = webdriver.Chrome('chromedriver.exe')
        # Open the specfied URL
        driver.get(url_ko)
        # Scrape sentences for each word present in the vocabulary list
        for word in words:
            # Sleep to avoid errors
            time.sleep(2)
            # Find the Search bar
            search = driver.find_element_by_xpath('//*[@id="ac_input"]')
            # Sleep to avoid errors
            time.sleep(1)
            # Type the word into the Search Box
            search.clear()
            # Sleep to avoid errors
            time.sleep(1)
            search.send_keys(word)
            search.send_keys(Keys.RETURN)
            # Sleep to avoid errors
            time.sleep(3)
            # Find the button titled "Examples" and click it
            example_button = driver.find_element_by_xpath('//*[@id="content"]/div[1]/div[1]/div/div/a[4]')
            example_button.send_keys(Keys.RETURN)
            # Sleep to avoid errors
            time.sleep(1)
            # Sleep to avoid errors
            time.sleep(1)
            # Find the "Have Translations" check box and check it
            have_translations = driver.find_element_by_xpath('//*[@id="searchPage_example"]/div[1]/div/span[5]/label').click()
            # Sleep to avoid errors
            time.sleep(2.5)
            # Sleep to avoid errors
            time.sleep(2)
            # Find the sentence examples
            examples = driver.find_element_by_xpath('//*[@id="searchPage_example"]/div[2]')
            # Define an object with all the example sentences
            unfiltered_sentences = examples.text
            pass1 = unfiltered_sentences.replace("발음듣기", "")
            pass2 = pass1.replace("Oxford Advanced Learner's English-Korean Dictionary", "")
            pass3 = pass2.replace("절대어휘 5100", "")
            pass4 = pass3.replace("국제영어대학원대학교 신어사전", "")
            pass5 = pass4.replace("DARAKWON", "")
            pass6 = pass5.replace("Friendict Level English-Korean Dictionary", "")
            pass7 = pass6.replace("Web Collection", "")
            pass8 = pass7.replace("YBM All in All English-Korean Dictionary", "")
            pass9 = pass8.replace("Neungyule", "")
            pass10 = pass9.replace("User translation", "")
            pass11 = pass10.replace("Hansard", "")
            pass12 = pass11.replace("영어영작문대사전", "")
            pass13 = pass12.replace("English Hidden Card", "")
            pass14 = pass13.replace("TIMES CORE", "")
            pass15 = pass14.replace("Dong-a's Prime English-Korean Dictionary", "")
            # Create a Dictionary with which to create the final DataFrame
            myList = pass15.split("\n\n")
            # Remove the parentheitcal words from the list
            myList2 = []
            for i in myList:
                i = re.sub(r'\([^()]*\)', '', i)
                i = i.lstrip()
                i = i.rstrip("\n")
                myList2.append(i)
            # Make dictionary from the new list
            myDict = dict(zip(myList2[::2], myList2[1::2]))
            english = myDict.keys()
            foreign = myDict.values()
            # Create a DataFrame from the scraped string
            df = pd.DataFrame(columns = ['English', 'Foreign', 'Word', 'Hint', 'Cloze'])
            df['English'] = english
            df['Foreign'] = foreign
            df['Word'] = word
            wordsClean = words.tolist()
            wordsClean = str(wordsClean).strip("[]")
            wordsClean = wordsClean.replace("'", "")
            df['Hint'] = wordsClean
            df['Hint'] = df['Hint'].str.replace("'", "")
            # Handle conjugations
            if "하다" in word:
                wordConj = word[:-2]
            elif "다" in word:
                wordConj = word[:-1]
            else:
                wordConj = word
            # Create the Cloze sentences    
            df['Cloze'] = df['Foreign'].str.replace(wordConj, '{{c1::' + wordConj + '}}')
            # Remove examples that don't contain a period
            df['English'] = df['English'].map(str)
            # Reduce the DataFrame to only 3 sentences
            df2 = df.iloc[:3]
            # Create the final DataFrame for the word
            w[word] = df2
            # Add the DataFrame to the list of all word DataFrames
            concatList.append(w[word])
            n += 1
            o -= 1
            print("Elapsed time: ({}) {} for {} words ({} remaining)".format(word, convert(time.time() - startTime1), n, o))
            # Exit the FOR loop
        # Quit the browser
        driver.quit()
        # Concatenate each word's DataFrame to the final one with all the sentences
        dfm = pd.concat(concatList, ignore_index = True)
        # Remove the pesky fuckers that got through the cleaning passes filters
        dfm['Foreign'] = dfm['Foreign'].str.replace('\n.*', '')
        dfm['Cloze'] = dfm['Cloze'].str.replace('\n.*', '')
        # return the created DataFrame
        return dfm
        # Exit the IF-ELIF statement
    elif language == '日本語':
        pass

# DEFINE FUNCTIONS

def convert(seconds): 
    seconds = seconds % (24 * 3600) 
    hour = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60
    return "%dh %02dm %02ds" % (hour, minutes, seconds)

def convertTTS(seconds): 
    seconds = seconds % (24 * 3600) 
    hour = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60
    return "%d hours %02d minutes %02d seconds" % (hour, minutes, seconds)

# Define the Anki-Connect functions
def request(action, **params):
    return {'action': action, 'params': params, 'version': 6}
def invoke(action, **params):
    requestJson = json.dumps(request(action, **params)).encode('utf-8')
    response = json.load(urllib.request.urlopen(urllib.request.Request('http://localhost:8765', requestJson)))
    if len(response) != 2:
        raise Exception('response has an unexpected number of fields')
    if 'error' not in response:
        raise Exception('response is missing required error field')
    if 'result' not in response:
        raise Exception('response is missing required result field')
    if response['error'] is not None:
        raise Exception(response['error'])
    return response['result']

# Define the Database Updating function
def updateDatabase(language):
    # Load the current Database into DataFrames
    wordsDF = pd.read_excel(r'Databases\{}_words.xlsx'.format(language), sheet_name = 'words', header = 0)
    sentencesDF =  pd.read_excel(r'Databases\{}_sentences.xlsx'.format(language), sheet_name = 'sentences', header = 0)
    # Get today's date and store it as a string
    date = datetime.now().strftime("%Y-%m-%d")
    if language != '日本語':  
        # Format the DataFrame to be appended to our local Database
        dfmAppend = dfm.drop(axis = 1, columns = ['Hint', 'Cloze'])
        # Name the columns in accordance with Flashcards.xlsx
        dfmAppend.columns = ['ENGLISH', 'FOREIGN', 'WORD']
        # Set the date of each row to today's date
        dfmAppend['DATE'] = date
    else:
        pass
    # Create the Vocabulary DataFrame that will be updated 
    if language != '日本語':
        vocabAppend = vocab
        vocabAppend.columns = ['FOREIGN', 'ENGLISH']
    else:
        vocabAppend = vocab
        vocabAppend.columns = ['FOREIGN', 'ENGLISH', 'KANJI']
    vocabAppend['DATE'] = date

    # Append the new rows to the DataFrames
    wordsDF = wordsDF.append(vocabAppend)
    
    # Create new XLSX files with including the appended rows
    wordsDF.to_excel('Databases\{}_words.xlsx'.format(language), sheet_name = 'words', index = False)

    if language != '日本語':
        sentencesDF = sentencesDF.append(dfmAppend)
        sentencesDF.to_excel('Databases\{}_sentences.xlsx'.format(language), sheet_name = 'sentences', index = False)
    else:
        pass

# THE REAL WORK BEGINS

# Read CSVs into DataFrames
vocab = pd.read_excel(r'Flashcards.xlsx', sheet_name = 'vocab', header = 0)
vocab.columns = ['Foreign', 'English', 'Kanji', 'Language']
vocab['English'] = vocab['English'].replace('s/\([^)]*\)//', '')
# Retrieve the language from the vocab DataFrame then drop its column
language = vocab['Language'].loc[0:0].to_string(index = False, header = False)
vocab.drop(['Language'], inplace = True, axis = 1)
# Strip the leading space from the lang variable (Thank you shit Excel dropdown menu! /s)
language = language.lstrip()
# Drop nan values in the Vocab DataFrame
if language == '日本語':
    vocab.dropna(axis = 0, inplace = True)
elif language == '中文' or '한국어': 
    vocab.drop(['Kanji'], inplace = True, axis = 1)
    vocab.dropna(axis = 0, inplace = True)

# Define the returned DataFrame from scrapeSentences()
words = vocab['Foreign']
print("Processing {} {} words for scraping...".format(words.shape[0], language))
dfm = scrapeSentences(language)
# Define the necessary variables
vocabForeign = vocab['Foreign'].to_list()
vocabEnglish = vocab['English'].to_list()
# Define Kanji if it exists
if 'Kanji' in vocab.columns:
    kanji = vocab['Kanji'].to_list()
else:
    pass
# Add the Notes
print("Processing {} {} Vocabulary...".format(words.shape[0], language))
addVocab(language)
# Define the necessary variables
if language != '日本語':
    cloze = dfm['Cloze'].to_list()
    tip = dfm['Hint'].to_list()
    english = dfm['English'].to_list()
    fullSentence = dfm['Foreign'].to_list()
    print("Processing {} {} Clozes...".format(dfm.shape[0], language))
    addCloze(language)
print("Updating {} Databases...".format(language))
updateDatabase(language)
print("ALL PROCESSES COMPLETED! ELAPSED TIME: {}".format(convert(time.time() - startTime1)))
tts = gTTS(text = "All processes have completed in {}.".format(convertTTS(time.time() - startTime1)), lang = "en")
tts.save("tts.mp3")
playsound("tts.mp3")
os.remove("tts.mp3")

Processing 13 中文 words for scraping...
Elapsed time: (骚扰) 0h 00m 11s for 1 words (12 remaining)
Elapsed time: (气不忿儿) 0h 00m 16s for 2 words (11 remaining)
Elapsed time: (煽动者) 0h 00m 22s for 3 words (10 remaining)
Elapsed time: (初衷) 0h 00m 27s for 4 words (9 remaining)
Elapsed time: (节约) 0h 00m 32s for 5 words (8 remaining)
Elapsed time: (助力) 0h 00m 38s for 6 words (7 remaining)
Elapsed time: (推动) 0h 00m 43s for 7 words (6 remaining)
Elapsed time: (制止) 0h 00m 49s for 8 words (5 remaining)
Elapsed time: (歉意) 0h 00m 54s for 9 words (4 remaining)
Elapsed time: (周全) 0h 00m 59s for 10 words (3 remaining)
Elapsed time: (完善) 0h 01m 05s for 11 words (2 remaining)
Elapsed time: (精准) 0h 01m 10s for 12 words (1 remaining)
Elapsed time: (搭配) 0h 01m 16s for 13 words (0 remaining)
Processing 13 中文 Vocabulary...
0h 01m 20s has elapsed for 1 Vocab words of 13 (12 remaining).
0h 01m 23s has elapsed for 2 Vocab words of 13 (11 remaining).
0h 01m 26s has elapsed for 3 Vocab words of 13 (10 remaining).
0h 

Exception: cannot create note because it is empty

In [7]:
updateDatabase(language)

## Add Clozes that were Duplicates

In [52]:
dfm

,English,Foreign,Word,Hint,Cloze
0,This man harasses his female co-workers.,这个人骚扰他的女同事。,骚扰,"骚扰, 气不忿儿, 煽动者, 初衷, 节约, 助力, 推动, 制止, 歉意, 周全, 完善,...",这个人{{c1::骚扰}}他的女同事。
1,She gave him hell at the slightest opportunity.,她抓住任何机会骚扰他。,骚扰,"骚扰, 气不忿儿, 煽动者, 初衷, 节约, 助力, 推动, 制止, 歉意, 周全, 完善,...",她抓住任何机会{{c1::骚扰}}他。
2,They were harassed in many ways.,他们受到了多种形式的骚扰。,骚扰,"骚扰, 气不忿儿, 煽动者, 初衷, 节约, 助力, 推动, 制止, 歉意, 周全, 完善,...",他们受到了多种形式的{{c1::骚扰}}。
3,We just can't stand despotic clauses like this.,对这种霸王条款，我们就是气不忿儿！,气不忿儿,"骚扰, 气不忿儿, 煽动者, 初衷, 节约, 助力, 推动, 制止, 歉意, 周全, 完善,...",对这种霸王条款，我们就是{{c1::气不忿儿}}！
4,We just can't stand despotic clauses like this.,对这种霸王条款，我们就是气不忿儿!,气不忿儿,"骚扰, 气不忿儿, 煽动者, 初衷, 节约, 助力, 推动, 制止, 歉意, 周全, 完善,...",对这种霸王条款，我们就是{{c1::气不忿儿}}!
5,I think these tires need some air.,我想这些轮胎要打点儿气。,气不忿儿,"骚扰, 气不忿儿, 煽动者, 初衷, 节约, 助力, 推动, 制止, 歉意, 周全, 完善,...",我想这些轮胎要打点儿气。
6,A pestilent agitator.,恶毒的煽动者,煽动者,"骚扰, 气不忿儿, 煽动者, 初衷, 节约, 助力, 推动, 制止, 歉意, 周全, 完善,...",恶毒的{{c1::煽动者}}
7,Tell me instantly he arrives.,"煽动者, 挑动者",煽动者,"骚扰, 气不忿儿, 煽动者, 初衷, 节约, 助力, 推动, 制止, 歉意, 周全, 完善,...","{{c1::煽动者}}, 挑动者"
8,Tell me instantly he arrives.,煽动者，挑动者。,煽动者,"骚扰, 气不忿儿, 煽动者, 初衷, 节约, 助力, 推动, 制止, 歉意, 周全, 完善,...",{{c1::煽动者}}，挑动者。
9,He bent from his original position and took an...,他改变初衷，站到了另一边。,初衷,"骚扰, 气不忿儿, 煽动者, 初衷, 节约, 助力, 推动, 制止, 歉意, 周全, 完善,...",他改变{{c1::初衷}}，站到了另一边。


In [48]:
sent = pd.read_excel(r'Databases\{}_sentences.xlsx'.format(language), sheet_name = 'sentences', header = 0)
start = 236
for index, row in sent.iterrows():
    sent['CLOZE'][index] = sent['FOREIGN'][index].replace(sent['WORD'][index], '{{c1::' + sent['WORD'][index] + '}}')
sentences = sent['CLOZE']
english = sent['ENGLISH'][start:].to_list()
tip = sent['WORD'][start:].to_list()

KeyError: 'CLOZE'

In [46]:
sent[start:]

,FOREIGN,ENGLISH,WORD,DATE,CLOZE
211,这个人骚扰他的女同事。,This man harasses his female co-workers.,骚扰,2020-12-04,这个人骚扰他的女同事。
212,她抓住任何机会骚扰他。,She gave him hell at the slightest opportunity.,骚扰,2020-12-04,她抓住任何机会骚扰他。
213,他们受到了多种形式的骚扰。,They were harassed in many ways.,骚扰,2020-12-04,他们受到了多种形式的骚扰。
214,对这种霸王条款，我们就是气不忿儿！,We just can't stand despotic clauses like this.,气不忿儿,2020-12-04,对这种霸王条款，我们就是气不忿儿！
215,对这种霸王条款，我们就是气不忿儿!,We just can't stand despotic clauses like this.,气不忿儿,2020-12-04,对这种霸王条款，我们就是气不忿儿!
216,恶毒的煽动者,A pestilent agitator.,煽动者,2020-12-04,恶毒的煽动者
217,"煽动者, 挑动者",Tell me instantly he arrives.,煽动者,2020-12-04,"煽动者, 挑动者"
218,煽动者，挑动者。,Tell me instantly he arrives.,煽动者,2020-12-04,煽动者，挑动者。
219,他改变初衷，站到了另一边。,He bent from his original position and took an...,初衷,2020-12-04,他改变初衷，站到了另一边。
220,成立这个联合会的初衷是为音乐家们服务。,The union came into existence to serve the int...,初衷,2020-12-04,成立这个联合会的初衷是为音乐家们服务。


In [37]:
addCloze(language)

Exception: cannot create note because it is a duplicate